In [ ]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm

In [ ]:
# infile = r"C:\git\msn_library\data\gnpslib\small_gnps.mgf"
# infile = r"C:\git\msn_library\data\masst\20230312_mce_library_pos_all_lib_MS2.mgf"
# infile = r"C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.mgf"
infile = r"C:\git\msn_library\data\gnpslib\library_cleanup\ALL_GNPS_NO_PROPOGATED.mgf"

In [ ]:
ids, inchi, smiles, inchikey, names  = [], [], [], [], []

with pyteomics.mgf.MGF(infile) as f_in:
    for spectrum_dict in tqdm(f_in):
        if (
            int(spectrum_dict["params"]["libraryquality"]) <= 3 and
            float(spectrum_dict["params"]["pepmass"][0]) > 0 and
            len(spectrum_dict["m/z array"]) >= 3 and
            (
                spectrum_dict["params"]["inchi"] != "N/A" or
                spectrum_dict["params"]["smiles"] != "N/A" or
                spectrum_dict["params"]["inchiaux"] != "N/A"
            )
        ):
            ids.append(spectrum_dict["params"]["spectrumid"])
            inchi.append(spectrum_dict["params"]["inchi"])
            inchikey.append(spectrum_dict["params"]["inchiaux"])
            smiles.append(spectrum_dict["params"]["smiles"])
            names.append(spectrum_dict["params"]["name"])

df = pd.DataFrame({"gnps_libid": ids, "inchi": inchi, "inchi_key": inchikey, "Smiles": smiles, "compound_name": names, })

# remove adduct from name
df["compound_name"] = [str(name).rsplit(" M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 2M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 3M+")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 2M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" 3M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" CollisionEnergy")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" [M-")[0] for name in df['compound_name']]
df["compound_name"] = [str(name).rsplit(" [M+")[0] for name in df['compound_name']]
df

In [ ]:
df.groupby(['inchi_key', 'inchi', 'Smiles', 'compound_name'], dropna=False)['gnps_libid'].apply(', '.join).reset_index()


In [ ]:
df = df.groupby(['inchi_key', 'inchi', 'Smiles', 'compound_name'])['gnps_libid'].apply(', '.join).reset_index()
df["entries"] = [str(lib).count(",") + 1 for lib in df["gnps_libid"]]

# df.to_csv("data/gnpslib/gnps_library.csv", index=False)
# df
df.shape

In [ ]:
df.loc[df[compound]]

In [ ]:
df = pd.read_csv("data/gnpslib/gnps_library.csv")

In [ ]:

inchikeys = """BGVLELSCIHASRV-QPEQYQDCSA-N
SWMDAPWAQQTBOG-UHFFFAOYSA-N
PZUSGRHVYDQLHR-UHFFFAOYSA-N
KPFZCKDPBMGECB-WGDLNXRISA-N
RYFZBPVMVYTEKZ-KBPBESRZSA-N
JEGHXKRHKHPBJD-UHFFFAOYSA-N
PCMHOSYCWRRHTG-UHFFFAOYSA-N
KFRKRECSIYXARE-HMAPJEAMSA-N
LHSBZAWDPSTOEY-UHFFFAOYSA-N
CYVVJSKZRBZHAV-UNZYHPAISA-N
ZGBAJMQHJDFTQJ-DEOSSOPVSA-N
IZAOBRWCUGOKNH-OAHLLOKOSA-N
HPHUVLMMVZITSG-UHFFFAOYSA-N
RDJGLLICXDHJDY-UHFFFAOYSA-N
RCYPVQCPYKNSTG-UHFFFAOYSA-N
SSZHESNDOMBSRV-UHFFFAOYSA-N
HRDQQHUKUIKFHT-UHFFFAOYSA-N
XGOYIMQSIKSOBS-UHFFFAOYSA-N
JNUGFGAVPBYSHF-UHFFFAOYSA-N
XDLYKKIQACFMJG-WKILWMFISA-N
KFRKRECSIYXARE-HYARGMPZSA-N""".split("\n")
import pubchempy
df = pubchempy.get_compounds(inchikeys, "inchikey", as_dataframe= True)
df

In [ ]:
df

In [ ]:

results_df = pubchempy.get_compounds(df[df["inchi_key"].notna()]["inchi_key"].head(50).tolist(), "inchikey", as_dataframe=False)
results_df

In [ ]:
comp = results_df[0]
compounds = [ {
    "inchi_key": comp.inchikey,
    "inchi": comp.inchi,
    "canonical_smiles": comp.canonical_smiles,
    "isomerical_smiles": comp.isomeric_smiles,
    "iupac": comp.iupac_name,
    "pubchem_cid": comp.cid,
} for comp in results_df]
resdf = pd.DataFrame(compounds)
resdf

In [ ]:
df.merge(resdf, how="left", on="inchi_key")

In [ ]:

infile = r"C:\git\msn_library\data\final_tables\gnps_drugs_clinical_phase_greater0.mgf"
spectra_counter = 0
with open(infile, 'r', encoding='UTF-8') as file:
    for line in tqdm(file):
        if line.lower().strip() == "begin ions":
            spectra_counter+=1
spectra_counter